In [6]:
import tensorflow as tf
import collections
import numpy as np

UNKNOWN_LABEL = "UNK"


def load_corpus(filename):
    corpus = []
    with open(filename) as infile:
        for line in infile:
            corpus.append(tf.compat.as_str(line.strip()).split())
    return corpus
            
def stream_corpus(filename):
    with open(filename) as infile:
        for line in infile:
            yield line.strip().split()


def compute_stats_and_fmap(corpus, min_shows):

    stats = {}
    for sent in corpus:
        for w in sent:
            stats[w] = stats.get(w, 0) + 1

    stats_filtered = {}
    fmap = {UNKNOWN_LABEL: 0}
    fid = 1
    for w in sorted(stats, key=stats.__getitem__, reverse=True):
        count = stats[w]
        if count >= min_shows:
            fmap[w] = fid
            fid += 1
            stats_filtered[w] = fid
    return stats_filtered, fmap


def process_corpus(corpus, min_shows=5, add_unk=True):
    stats, fmap = compute_stats_and_fmap(corpus, min_shows)
    dataset = []
    for sent in corpus:
        current_sent = []
        for w in sent:
            if w in fmap:
                current_sent.append(fmap[w])
            else:
                if add_unk:
                    current_sent.append(fmap[UNKNOWN_LABEL])
        dataset.append(current_sent)

    reversed_fmap = dict((fid, word) for word, fid in fmap.iteritems())
    return dataset, stats, fmap, reversed_fmap



def skipgram_sent_streamer(corpus_iter, context_window):
    for sent in corpus_iter:
        for central, w in enumerate(sent):
            #print central, w
            for context in xrange(
                            max(0, central - context_window), 
                            min(len(sent), central + context_window + 1)
                            ):
                if context == central:
                    continue
                wc = sent[context]
                yield (w, wc)


class BatchIter(object):
    def __init__(self, corpus_iter, context_window, batch_size,
                streamer="sent"):
        self._corpus_iter = corpus_iter # corpus_iter returns sentences
        self._batch_size = batch_size
        #self._context_window = context_window
        
        if streamer == "sent":
            self._skipstreamer = skipgram_sent_streamer(corpus_iter, context_window)
        else:
            raise NotImplementedError
        
        # batch contains central words 
        # labels contains context words
        #self._next_batch = np.ndarray(batch_size, dtype=np.int32)
        #self._next_labels = np.ndarray((batch_size, 1), dtype=np.int32)
        #self._next_batch_examples = 0
        
    def __iter__(self):
        return self
    
    def next(self):
        
        batch_examples = 0        
        #central = np.ndarray(self._batch_size, dtype=np.int32)
        #labels = np.ndarray((self._batch_size, 1), dtype=np.int32)
        central = []
        context = []
        while batch_examples < self._batch_size:

            try:
                w, c = next(self._skipstreamer)
            except StopIteration:
                if context:
                    break
                else:
                    raise StopIteration
              
            #print w, c
            central.append(w)
            context.append(c)
            batch_examples += 1
        
        if batch_examples < self._batch_size:
            return None, None
        
        return np.asarray(central, dtype=np.int32), np.asarray(context, dtype=np.int32).reshape([batch_examples, 1])


In [7]:
corpus = [
    "1 2 3 4 5".split(),
    "3 2 1".split(),
    "1 2 1".split()
    ]

pairs = [pair for pair in skipgram_sent_streamer(corpus, 1)]
print pairs[:10]


batch_iter = BatchIter(corpus, 1, 5)
for b, l in batch_iter:
    print "batch: ", b, l


[('1', '2'), ('2', '1'), ('2', '3'), ('3', '2'), ('3', '4'), ('4', '3'), ('4', '5'), ('5', '4'), ('3', '2'), ('2', '3')]
batch:  [1 2 2 3 3] [[2]
 [1]
 [3]
 [2]
 [4]]
batch:  [4 4 5 3 2] [[3]
 [5]
 [4]
 [2]
 [3]]
batch:  [2 1 1 2 2] [[1]
 [2]
 [2]
 [1]
 [1]]
batch:  None None


In [8]:
filename = "ted_sentences_clean.txt"
corpus = load_corpus(filename)
dataset, stats, fmap, reversed_fmap = process_corpus(corpus)
print "sentences in dataset %d" % len(dataset)
vocabulary_size = len(fmap)
print "vocabulary_size % d" % vocabulary_size

corpus_iter = dataset
print dataset[:2]

sentences in dataset 266694
vocabulary_size  21445
[[70, 19, 103, 789, 553, 2035, 16, 119, 34, 61, 4, 1, 137, 48, 16, 119, 34, 22, 13, 108], [3, 52, 1, 223, 223, 787, 3, 843, 628, 12, 3506, 53, 1, 1672, 201, 103, 2258, 2269, 2, 6044]]


In [10]:
import tensorflow as tf
import numpy as np
import math


# Create computational graph
tf.reset_default_graph()

vocabulary_size = vocabulary_size
batch_size = 128
embedding_size = 100  # Dimension of the embedding vector.
skip_window = 1       # How many words to consider left and right.
num_skips = 2         # How many times to reuse an input to generate a label.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_neg_samples = 64    # Number of negative examples to sample.

graph = tf.Graph()
with graph.as_default():
    
  # Input data.
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

    # Ops and variables pinned to the CPU because of missing GPU implementation
    with tf.device('/cpu:0'):
        # Look up embeddings for inputs.
        embeddings = tf.Variable(
            tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
        embed = tf.nn.embedding_lookup(embeddings, train_inputs)

    # Construct the variables for the NCE loss
    nce_weights = tf.Variable(
        tf.truncated_normal([vocabulary_size, embedding_size],
                            stddev=1.0 / math.sqrt(embedding_size)))
    nce_biases = tf.Variable(tf.zeros([vocabulary_size]))
     
    # Compute the average NCE loss for the batch.
    # tf.nce_loss automatically draws a new sample of the negative labels each
    # time we evaluate the loss.
    loss = tf.reduce_mean(
      tf.nn.nce_loss(weights=nce_weights,
                     biases=nce_biases,
                     labels=train_labels,
                     inputs=embed,
                     num_sampled=num_neg_samples,
                     num_classes=vocabulary_size)
    )

    
    # Construct the SGD optimizer using a learning rate of 1.0.
    optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

    # Compute the cosine similarity between minibatch examples and all embeddings.
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
    similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)

    # Add variable initializer.
    init = tf.global_variables_initializer()

In [11]:
num_passes = 1

# Propagate data through graph
with tf.Session(graph=graph, config=tf.ConfigProto(log_device_placement=True)) as session:
    # We must initialize all variables before we use them.
    init.run()
    print('Initialized')

    average_loss = 0
    num_batch = 0
    for step in xrange(num_passes):
        batch_iter = BatchIter(corpus_iter, skip_window, batch_size)
        
        print step
        for batch_inputs, batch_labels in batch_iter:
            if batch_inputs is None:
                continue
            
            num_batch += 1
            feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

            # We perform one update step by evaluating the optimizer op (including it
            # in the list of returned values for session.run()
            _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
            average_loss += loss_val

            if num_batch % 2000 == 0:
                if num_batch > 0:
                    average_loss /= 2000

                # The average loss is an estimate of the loss over the last 2000 batches.
                print('Average loss at step ', num_batch, ': ', average_loss)
                average_loss = 0

            # Note that this is expensive (~20% slowdown if computed every 500 steps)
            
            if step < 0:
                sim = similarity.eval()

                for i in xrange(valid_size):
                    valid_word = reversed_fmap[valid_examples[i]]
                    top_k = 8  # number of nearest neighbors
                    nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                    log_str = 'Nearest to %s:' % valid_word
                    for k in xrange(top_k):
                        close_word = reversed_fmap[nearest[k]]
                        log_str = '%s %s,' % (log_str, close_word)
                        print(log_str)

    final_embeddings = normalized_embeddings.eval()

    
    
    
    
    
    
    

Initialized
0
('Average loss at step ', 2000, ': ', 71.407319506883624)
('Average loss at step ', 4000, ': ', 21.248210202455521)
('Average loss at step ', 6000, ': ', 11.714248725175857)
('Average loss at step ', 8000, ': ', 8.013604721546173)
('Average loss at step ', 10000, ': ', 6.6733183240890499)
('Average loss at step ', 12000, ': ', 6.056646643519402)
('Average loss at step ', 14000, ': ', 5.7205630761384967)
('Average loss at step ', 16000, ': ', 5.3874320895671843)
('Average loss at step ', 18000, ': ', 5.3299431227445604)
('Average loss at step ', 20000, ': ', 5.1454075814485547)
('Average loss at step ', 22000, ': ', 4.97230595433712)
('Average loss at step ', 24000, ': ', 4.9322701076269153)
('Average loss at step ', 26000, ': ', 4.8976795167922971)
('Average loss at step ', 28000, ': ', 4.8323580920696259)
('Average loss at step ', 30000, ': ', 4.8033932183980941)
('Average loss at step ', 32000, ': ', 4.7404894587993622)
('Average loss at step ', 34000, ': ', 4.717061794